# Your first scraper
In this project, we will guide you step by step through the process of:

1. creating a self-contained development environment.
1. retrieving some information from an API (a website for computers)
2. leveraging it to scrape a website that does not provide an API
3. saving the output for later processing

Here we query an API for a list of countries and their past leaders. We then extract and sanitize their short bio from Wikipedia. Finally, we save the data to disk.

This task is often the first (coding) step of a datascience project and you will often come back to it in the future.

You will study topics such as *scraping*, *data structures*, *regular expressions*, *concurrency* and *file handling*. We will point out useful resources at the appropriate time. 

Let's dive in!

## Creating a clean environment

Use the [`venv`](https://docs.python.org/3/library/venv.html) command to create a new environment called `wikipedia_scraper_env`.

Activate it.

You will find more info about virtual environments in the course content and on the web.

## API Scraping

### A simple API query
You will start with the basics: how to do a simple request to an [API endpoint](../../2.python/2.python_advanced/05.Scraping/5.apis.ipynb).

You will use the [requests](https://requests.readthedocs.io/en/latest/) external library through the `import` keyword.

NOTE: external libraries need to be installed first. Check their documentation.

Check the [Quickstart](https://requests.readthedocs.io/en/latest/user/quickstart/) section of the Requests documentation to:
1. use the `get()` method to connect to this endpoint: https://country-leaders.onrender.com/status
2. check if the `status_code` is equal to 200, which means OK.
    * if OK, `print()` the `text`` of the response.
    * if not, `print()` the `status_code`. 

Here is the signification of [HTTP status codes](https://en.wikipedia.org/wiki/List_of_HTTP_status_codes).


In [ ]:
import requests
root_url = 'https://country-leaders.onrender.com/'
status_url = 'https://country-leaders.onrender.com/status'
req = requests.get(status_url)
if req.status_code == 200:
    print(req)
elif req.status_code != 200:
    print(req.status_code)

<Response [200]>


### Dealing with JSON

[JSON](https://quickref.me/json) is the preferred format to deal with data over the web. You cannot avoid it so you would better get acquainted.

Connect to another endpoint called `/countries` but this time the API will return data in the JSON format. Check what to do in the Requests [Quickstart](https://requests.readthedocs.io/en/latest/user/quickstart/).


In [151]:
countries_url = 'https://country-leaders.onrender.com/countries'
req = requests.get(countries_url)
countries = req.json()
countries


{'message': 'The cookie is missing'}

### Cookies anyone?

It looks like the access to this API is restricted...
Query the `/cookie` endpoint and extract the appropriate field to access your cookie.

You will need to use this cookie in each of the following API requests.

In [152]:
def cookie():
    cookie_url = 'https://country-leaders.onrender.com/cookie'
    req_cookie = requests.get(cookie_url)
    return req_cookie.cookies
cookie()

<RequestsCookieJar[Cookie(version=0, name='user_cookie', value='9a107119-5907-4831-b7d2-a8e71d3da3b6', port=None, port_specified=False, domain='country-leaders.onrender.com', domain_specified=False, domain_initial_dot=False, path='/', path_specified=True, secure=False, expires=None, discard=True, comment=None, comment_url=None, rest={'SameSite': 'lax'}, rfc2109=False)]>

Try to query the countries endpoint using the cookie, save the output and print it.

In [153]:
cookies = cookie()
req = requests.get(countries_url, cookies=cookies)
countries=req.json()
countries


['us', 'be', 'ma', 'ru', 'fr']

Chances are the cookie has expired... Thanksfully, you got a nice error message. For now, simply execute the last 2 cells quickly so you get a result.

### Getting the actual data from the API

Query the `/leaders` endpoint.

In [154]:
cookies = cookie()
leaders_url = 'https://country-leaders.onrender.com/leaders'
leaders = requests.get(leaders_url, cookies=cookies).json()
leaders

{'message': 'Please specify a country'}

It looks like this endpoint requires additional information in order to return its result. Check the API [*documentation*](https://country-leaders.onrender.com/docs) in your web browser.

Change the query to accept *parameters*. You should know where to find help by now.

In [155]:
cookies = cookie()
for x in countries:
    leaders_url = 'https://country-leaders.onrender.com/leaders'
    leaders = requests.get(leaders_url, cookies=cookies, params={'country': x}).json()
    print(leaders)


[{'id': 'Q23', 'first_name': 'George', 'last_name': 'Washington', 'birth_date': '1732-02-22', 'death_date': '1799-12-14', 'place_of_birth': 'Westmoreland County', 'wikipedia_url': 'https://en.wikipedia.org/wiki/George_Washington', 'start_mandate': '1789-04-30', 'end_mandate': '1797-03-04'}, {'id': 'Q76', 'first_name': 'Barack', 'last_name': 'Obama', 'birth_date': '1961-08-04', 'death_date': None, 'place_of_birth': 'Kapiolani Medical Center for Women and Children', 'wikipedia_url': 'https://en.wikipedia.org/wiki/Barack_Obama', 'start_mandate': '2009-01-20', 'end_mandate': '2017-01-20'}, {'id': 'Q91', 'first_name': 'Abraham', 'last_name': 'Lincoln', 'birth_date': '1809-02-12', 'death_date': '1865-04-15', 'place_of_birth': 'Sinking Spring Farm', 'wikipedia_url': 'https://en.wikipedia.org/wiki/Abraham_Lincoln', 'start_mandate': '1861-03-04', 'end_mandate': '1865-04-15'}, {'id': 'Q207', 'first_name': 'George', 'last_name': 'Bush', 'birth_date': '1946-07-06', 'death_date': None, 'place_of_bi

### A sneak peak at the data (finally)

Look inside a few examples. Notice the dictionary keys available for each entry. You have your first example of *structured data*. This data was sanitized for your benefit, meaning it is readily exploitable without modification.

You will also notice there is a Wikipedia link for each entry. You will need to extract additional information there. This will be a case of *semi-structured* data.

The /countries endpoint returns a `list` of several country codes.

You need to loop through this list and query the /leaders endpoint for each one. Save each `json` result in a dictionary called `leaders_per_country`.

In [156]:
leaders_per_country = {}
cookies = cookie()
for x in countries:
    leaders_url = 'https://country-leaders.onrender.com/leaders'
    leaders = requests.get(leaders_url, cookies=cookies, params={'country': x}).json()
    leaders_per_country[x] = leaders


In [157]:
print(leaders_per_country)

{'us': [{'id': 'Q23', 'first_name': 'George', 'last_name': 'Washington', 'birth_date': '1732-02-22', 'death_date': '1799-12-14', 'place_of_birth': 'Westmoreland County', 'wikipedia_url': 'https://en.wikipedia.org/wiki/George_Washington', 'start_mandate': '1789-04-30', 'end_mandate': '1797-03-04'}, {'id': 'Q76', 'first_name': 'Barack', 'last_name': 'Obama', 'birth_date': '1961-08-04', 'death_date': None, 'place_of_birth': 'Kapiolani Medical Center for Women and Children', 'wikipedia_url': 'https://en.wikipedia.org/wiki/Barack_Obama', 'start_mandate': '2009-01-20', 'end_mandate': '2017-01-20'}, {'id': 'Q91', 'first_name': 'Abraham', 'last_name': 'Lincoln', 'birth_date': '1809-02-12', 'death_date': '1865-04-15', 'place_of_birth': 'Sinking Spring Farm', 'wikipedia_url': 'https://en.wikipedia.org/wiki/Abraham_Lincoln', 'start_mandate': '1861-03-04', 'end_mandate': '1865-04-15'}, {'id': 'Q207', 'first_name': 'George', 'last_name': 'Bush', 'birth_date': '1946-07-06', 'death_date': None, 'plac

It is finally time to create a `get_leaders()` function for the above code. You will build on it later-on. This function takes no parameter. Inside it, you will need to:
1. define the urls
2. get the cookies
2. get the countries
3. loop over them and save their leaders in a dictionary
4. return the dictionary

In [158]:
import requests
def get_leaders():
    leaders_dict = {}
    root_url = 'https://country-leaders.onrender.com/'
    status_url = 'https://country-leaders.onrender.com/status'
    cookie_url = 'https://country-leaders.onrender.com/cookie'
    countries_url = 'https://country-leaders.onrender.com/countries'
    leaders_url = 'https://country-leaders.onrender.com/leaders'
    req_cookie = requests.get(cookie_url)
    cookies = req_cookie.cookies
    req = requests.get(countries_url, cookies=cookies)
    countries = req.json()
    for i in countries:
        req_leaders = requests.get(leaders_url, cookies=cookies, params={'country': i}).json()
        leaders_dict[i] = req_leaders
    return leaders_dict



Test your function, save the result in the `leaders_per_country` dictionary and check its ouput.

In [159]:
leaders_per_country = get_leaders()
print(type(leaders_per_country))

<class 'dict'>


## Extracting data from Wikipedia

Query one of the leaders' Wikipedia urls and display its `text` (not JSON).

In [160]:
req_wiki = requests.get('https://en.wikipedia.org/wiki/George_Washington').text
print(req_wiki)

<!DOCTYPE html>
<html class="client-nojs vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-content-enabled vector-feature-custom-font-size-clientpref-1 vector-feature-appearance-pinned-clientpref-1 vector-feature-night-mode-enabled skin-theme-clientpref-day vector-toc-available" lang="en" dir="ltr">
<head>
<meta charset="UTF-8">
<title>George Washington - Wikipedia</title>
<script>(function(){var className="client-js vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector

Ouch! You get the raw HTML code of the webpage. If you try to deal with it without tools, you will be there all night. Instead, use the [beautiful soup 4](https://www.crummy.com/software/BeautifulSoup/bs4/doc/) *external* library. You will find more info about it [here](../../2.python/2.python_advanced/05.Scraping/1.beautifulsoup_basic.ipynb) and [here](../../2.python/2.python_advanced/05.Scraping/2.beautifulsoup_advanced.ipynb)

Using the Quickstart section, start by importing the library and loading the output of your `get_text()` function.

Use the `prettify()` function and print it to take a look. You will start the actual parsing in the next step.

In [161]:
from bs4 import BeautifulSoup as bs
req_wiki = requests.get('https://fr.wikipedia.org/wiki/Fran%C3%A7ois_Hollande').text 
soup = bs(req_wiki, 'html.parser')
soup.get_text()
print(soup.prettify())

<!DOCTYPE html>
<html class="client-nojs vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-content-enabled vector-feature-custom-font-size-clientpref-1 vector-feature-appearance-pinned-clientpref-1 vector-feature-night-mode-enabled skin-theme-clientpref-day vector-toc-available" dir="ltr" lang="fr">
 <head>
  <meta charset="utf-8"/>
  <title>
   François Hollande — Wikipédia
  </title>
  <script>
   (function(){var className="client-js vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-

That looks better but you need to extract the right part of the webpage: the text of the first paragraph.

It is a bit tricky because Wikipedia pages slightly differ in structure from one language to the next. We cannot simply get the text for the first HTML paragraph.

You will start by getting all the HTML paragraphs from the HTML source and saving them in the `paragraphs` variable.

Use the documentation or google the appropriate keywords.

In [162]:
paragraphs = []
for para in soup.find_all("p"): 
    paragraphs.append(para.get_text()) 
print(paragraphs)

['\n', 'Pour les articles homonymes, voir Hollande (homonymie).\n', '\n', "François Hollande [fʁɑ̃swa ɔlɑ̃d][n 3] Écouter, né le 12 août 1954 à Rouen (Seine-Inférieure), est un haut fonctionnaire et homme d'État français. Il est président de la République française du 15 mai 2012 au 14 mai 2017.\n", "Magistrat à la Cour des comptes et brièvement avocat, il est élu pour la première fois député en 1988. Il exerce la fonction de premier secrétaire du Parti socialiste (PS) de 1997 à 2008, pendant la troisième cohabitation puis dans l'opposition. Au niveau local, il est maire de Tulle de 2001 à 2008 et président du conseil général de Corrèze de 2008 à 2012.\n", "Il est désigné candidat du PS à l'élection présidentielle de 2012 à l'issue d'une primaire à gauche face à Martine Aubry, et élu chef de l'État face au président sortant, Nicolas Sarkozy, avec 51,6\xa0% des suffrages exprimés au second tour.\n", "Sa politique économique connaît un virage social-libéral en 2014, année du «\xa0pacte d

If you try different urls, you might find that the paragraph you want may be at a different index each time.

That is where you need to be clever and ask yourself what would be a reliable way to identify the right index ie. which string matches only the first paragraph whatever the language...

Spend a good 30 minutes on the problem and brainstorm with your fellow learners. If you come out empty handed, ask your coach.

1. Loop over the HTML paragraphs
2. When you have identified the correct one
  * store the [text](https://www.crummy.com/software/BeautifulSoup/bs4/doc/#output) inside the `first_paragraph` variable
  * exit the loop

In [163]:
paragraphs = []
for para in soup.find_all("p"): 
    if len(para)>8:
        paragraphs.append(para.get_text())
        print(paragraphs)
        break 


["François Hollande [fʁɑ̃swa ɔlɑ̃d][n 3] Écouter, né le 12 août 1954 à Rouen (Seine-Inférieure), est un haut fonctionnaire et homme d'État français. Il est président de la République française du 15 mai 2012 au 14 mai 2017.\n"]


In [164]:
wikipedia_links = []
for i in countries:   
    for president in leaders_per_country[i]:
        wikipedia_links.append(president['wikipedia_url'])
print(wikipedia_links)

['https://en.wikipedia.org/wiki/George_Washington', 'https://en.wikipedia.org/wiki/Barack_Obama', 'https://en.wikipedia.org/wiki/Abraham_Lincoln', 'https://en.wikipedia.org/wiki/George_W._Bush', 'https://en.wikipedia.org/wiki/Bill_Clinton', 'https://en.wikipedia.org/wiki/Joe_Biden', 'https://en.wikipedia.org/wiki/Franklin_D._Roosevelt', 'https://en.wikipedia.org/wiki/Andrew_Johnson', 'https://en.wikipedia.org/wiki/Gerald_Ford', 'https://en.wikipedia.org/wiki/Richard_Nixon', 'https://en.wikipedia.org/wiki/Lyndon_B._Johnson', 'https://en.wikipedia.org/wiki/John_F._Kennedy', 'https://en.wikipedia.org/wiki/Dwight_D._Eisenhower', 'https://en.wikipedia.org/wiki/Ronald_Reagan', 'https://en.wikipedia.org/wiki/Harry_S._Truman', 'https://en.wikipedia.org/wiki/John_Adams', 'https://en.wikipedia.org/wiki/Thomas_Jefferson', 'https://en.wikipedia.org/wiki/James_Madison', 'https://en.wikipedia.org/wiki/James_Monroe', 'https://en.wikipedia.org/wiki/John_Quincy_Adams', 'https://en.wikipedia.org/wiki/An

At this stage, you can create a function to maintain consistency in your code. We will give you its *skeleton*, you will copy the code you wrote and make it work inside a function.

Don't forget to test your function.

In [165]:
# 10 lines
def get_first_paragraph(wikipedia_url):
    print(wikipedia_url) # keep this for the rest of the notebook
    first_paragraph = ''
    req_wiki = requests.get(wikipedia_url).text 
    soup = bs(req_wiki, 'html.parser')
    for para in soup.find_all("p"): 
        if len(para)>8:
            first_paragraph = (para.get_text())
            #first_paragraph.append(para.get_text())
            break
    return first_paragraph
answer = {}
for i in wikipedia_links:
    answer[i] = get_first_paragraph(i)

#   return first_paragraph

https://en.wikipedia.org/wiki/George_Washington
https://en.wikipedia.org/wiki/Barack_Obama
https://en.wikipedia.org/wiki/Abraham_Lincoln
https://en.wikipedia.org/wiki/George_W._Bush
https://en.wikipedia.org/wiki/Bill_Clinton
https://en.wikipedia.org/wiki/Joe_Biden
https://en.wikipedia.org/wiki/Franklin_D._Roosevelt
https://en.wikipedia.org/wiki/Andrew_Johnson
https://en.wikipedia.org/wiki/Gerald_Ford
https://en.wikipedia.org/wiki/Richard_Nixon
https://en.wikipedia.org/wiki/Lyndon_B._Johnson
https://en.wikipedia.org/wiki/John_F._Kennedy
https://en.wikipedia.org/wiki/Dwight_D._Eisenhower
https://en.wikipedia.org/wiki/Ronald_Reagan
https://en.wikipedia.org/wiki/Harry_S._Truman
https://en.wikipedia.org/wiki/John_Adams
https://en.wikipedia.org/wiki/Thomas_Jefferson
https://en.wikipedia.org/wiki/James_Madison
https://en.wikipedia.org/wiki/James_Monroe
https://en.wikipedia.org/wiki/John_Quincy_Adams
https://en.wikipedia.org/wiki/Andrew_Jackson
https://en.wikipedia.org/wiki/Martin_Van_Buren
ht

### Regular expressions to the rescue

Now that you have extracted the content of the first paragraph, the only thing that remains to finish your Wikipedia scraper is to sanitize the output.

Indeed some Wikipedia references, HTML code, phonetic pronunciation etc. may linger. You might find *regular expressions* handy to get rid of them and obtain pristine text. You will find some useful documentation about regular expressions [here](../../2.python/2.python_advanced/03.Regex/regex.ipynb)

Once you have one of your regex working online, try it in the cell below. 

Hints: 
* Check the `sub()` method documentation.
* Make sure to test urls in different languages. Some may look good but other do not.

In [166]:
text = []
for key, value in answer.items():
    text.append(value)
print(text)


['George Washington (February 22, 1732\xa0– December 14, 1799) was a Founding Father of the United States, military officer, and farmer who served as the first president of the United States from 1789 to 1797. Appointed by the Second Continental Congress as commander of the Continental Army in 1775, Washington led Patriot forces to victory in the American Revolutionary War and then served as president of the Constitutional Convention in 1787, which drafted the current Constitution of the United States. Washington has thus become commonly known as the "Father of his Country".\n', 'Barack Hussein Obama II[a] (born August 4, 1961) is an American politician who served as the 44th president of the United States from 2009 to 2017. As a member of the Democratic Party, he was the first African-American president in U.S. history. Obama previously served as a U.S. senator representing Illinois from 2005 to 2008 and as an Illinois state senator from 1997 to 2004. \n', 'Abraham Lincoln (/ˈlɪŋkən/ 

In [180]:
import re

for i in text:
    i = re.sub(r'\[.*?\]', '', i)
    i = re.sub(r'\n', ' ', i)
    i = re.sub(r'ⓘ', '', i)
    i = re.sub(r'/.*?/', '', i)
    i = re.sub(r'\( *;? *Écouter *\)', '', i)
    i = re.sub(r'Écouter', '', i)
    i = re.sub(r' +', ' ', i)
    i = re.sub(r' ,', ',', i)
    print(i)    




George Washington (February 22, 1732 – December 14, 1799) was a Founding Father of the United States, military officer, and farmer who served as the first president of the United States from 1789 to 1797. Appointed by the Second Continental Congress as commander of the Continental Army in 1775, Washington led Patriot forces to victory in the American Revolutionary War and then served as president of the Constitutional Convention in 1787, which drafted the current Constitution of the United States. Washington has thus become commonly known as the "Father of his Country". 
Barack Hussein Obama II (born August 4, 1961) is an American politician who served as the 44th president of the United States from 2009 to 2017. As a member of the Democratic Party, he was the first African-American president in U.S. history. Obama previously served as a U.S. senator representing Illinois from 2005 to 2008 and as an Illinois state senator from 1997 to 2004. 
Abraham Lincoln ( LINK-ən; February 12, 1809

Overwrite the `get_first_paragraph()` function by applying your regex to the first paragraph before returning it.

In [191]:
def get_first_paragraph(wikipedia_url):
    print(wikipedia_url) # keep this for the rest of the notebook
    first_paragraph = ''
    req_wiki = requests.get(wikipedia_url).text 
    soup = bs(req_wiki, 'html.parser')
    for para in soup.find_all("p"): 
        if len(para)>8:
            first_paragraph = (para.get_text())
            #first_paragraph.append(para.get_text())
            break
    re.sub(r'\[.*?\]', '', first_paragraph)    
    return first_paragraph
get_first_paragraph('https://fr.wikipedia.org/wiki/Napol%C3%A9on_III')

https://fr.wikipedia.org/wiki/Napol%C3%A9on_III


"Napoléon\xa0III, né le 20 avril 1808 à Paris et mort le 9 janvier 1873 à Chislehurst (Royaume-Uni), est un monarque et homme d'État français. Il est à la fois sous son nom de Louis-Napoléon Bonaparte l'unique président de la Deuxième République, le premier chef d'État français élu au suffrage universel masculin, le 10 décembre 1848, le premier président de la République française et après la proclamation de l'Empire le 2 décembre 1852, le dernier monarque du pays en tant qu'empereur des Français.\n"

Come up with other regexes to capture other patterns and sanitize the outputs completely. Modify your `get_first_paragraph()` function accordingly.

In [239]:
def get_first_paragraph(wikipedia_url):
    print(wikipedia_url)
    first_paragraph = ''
    req_wiki = requests.get(wikipedia_url).text 
    soup = bs(req_wiki, 'html.parser')
    for para in soup.find_all("p"): 
        if len(para)>8:
            first_paragraph = (para.get_text())
            #first_paragraph.append(para.get_text())
            break
    first_paragraph = re.sub(r'\[.*?\]', '', first_paragraph)
    first_paragraph = re.sub(r'\[.*?\]', '', first_paragraph)
    first_paragraph = re.sub(r'\n', ' ', first_paragraph)
    first_paragraph = re.sub(r'ⓘ', '', first_paragraph)
    first_paragraph = re.sub(r'/.*?/', '', first_paragraph)
    first_paragraph = re.sub(r'\( *;? *Écouter *\)', '', first_paragraph)
    first_paragraph = re.sub(r'Écouter', '', first_paragraph)
    first_paragraph = re.sub(r' +', ' ', first_paragraph)
    first_paragraph = re.sub(r' ,', ',', first_paragraph)
    return first_paragraph
'''
answer = {}
for i in wikipedia_links:
    answer[i] = get_first_paragraph(i)
for key, value in answer.items():
    print(value)
'''


'\nanswer = {}\nfor i in wikipedia_links:\n    answer[i] = get_first_paragraph(i)\nfor key, value in answer.items():\n    print(value)\n'

## Putting it all together

Let's go back to your `get_leaders()` function and update it with an *inner* loop over each leader. You will query the url provided and extract the first paragraph using the `get_first_paragraph()` function you just finished. You will then update that `leader`'s dictionary and move on to the next one.

Notice, the rest of the code should not change since you modify the leader's data one by one.

In [237]:
def get_leaders():
    leaders_dict = {}
    root_url = 'https://country-leaders.onrender.com/'
    status_url = 'https://country-leaders.onrender.com/status'
    cookie_url = 'https://country-leaders.onrender.com/cookie'
    countries_url = 'https://country-leaders.onrender.com/countries'
    leaders_url = 'https://country-leaders.onrender.com/leaders'
    req_cookie = requests.get(cookie_url)
    cookies = req_cookie.cookies
    req = requests.get(countries_url, cookies=cookies)
    countries = req.json()
    for i in countries:
        req_leaders = requests.get(leaders_url, cookies=cookies, params={'country': i}).json()
        leaders_dict[i] = req_leaders
    for i in countries:
        for leader in leaders_dict[i]:
            wikipedia_url = leader['wikipedia_url']
            first_paragraph = get_first_paragraph(wikipedia_url)
            leader['first_paragraph'] = first_paragraph

    return leaders_dict


In [240]:
get_leaders()

https://en.wikipedia.org/wiki/George_Washington
https://en.wikipedia.org/wiki/Barack_Obama
https://en.wikipedia.org/wiki/Abraham_Lincoln
https://en.wikipedia.org/wiki/George_W._Bush
https://en.wikipedia.org/wiki/Bill_Clinton
https://en.wikipedia.org/wiki/Joe_Biden
https://en.wikipedia.org/wiki/Franklin_D._Roosevelt
https://en.wikipedia.org/wiki/Andrew_Johnson
https://en.wikipedia.org/wiki/Gerald_Ford
https://en.wikipedia.org/wiki/Richard_Nixon
https://en.wikipedia.org/wiki/Lyndon_B._Johnson
https://en.wikipedia.org/wiki/John_F._Kennedy
https://en.wikipedia.org/wiki/Dwight_D._Eisenhower
https://en.wikipedia.org/wiki/Ronald_Reagan
https://en.wikipedia.org/wiki/Harry_S._Truman
https://en.wikipedia.org/wiki/John_Adams
https://en.wikipedia.org/wiki/Thomas_Jefferson
https://en.wikipedia.org/wiki/James_Madison
https://en.wikipedia.org/wiki/James_Monroe
https://en.wikipedia.org/wiki/John_Quincy_Adams
https://en.wikipedia.org/wiki/Andrew_Jackson
https://en.wikipedia.org/wiki/Martin_Van_Buren
ht

{'us': [{'id': 'Q23',
   'first_name': 'George',
   'last_name': 'Washington',
   'birth_date': '1732-02-22',
   'death_date': '1799-12-14',
   'place_of_birth': 'Westmoreland County',
   'wikipedia_url': 'https://en.wikipedia.org/wiki/George_Washington',
   'start_mandate': '1789-04-30',
   'end_mandate': '1797-03-04',
   'first_paragraph': 'George Washington (February 22, 1732\xa0– December 14, 1799) was a Founding Father of the United States, military officer, and farmer who served as the first president of the United States from 1789 to 1797. Appointed by the Second Continental Congress as commander of the Continental Army in 1775, Washington led Patriot forces to victory in the American Revolutionary War and then served as president of the Constitutional Convention in 1787, which drafted the current Constitution of the United States. Washington has thus become commonly known as the "Father of his Country". '},
  {'id': 'Q76',
   'first_name': 'Barack',
   'last_name': 'Obama',
   

Does the function crash in the middle of the loop? Chances are the cookies have expired while looping over the leaders.

Modify your function with an *exception* or check if the `status_code` is a cookie error. In either case, get new cookies and query the api again.

If your code did not crash,

In [241]:
# < 25 lines
def get_leaders():
    leaders_dict = {}
    root_url = 'https://country-leaders.onrender.com/'
    status_url = 'https://country-leaders.onrender.com/status'
    cookie_url = 'https://country-leaders.onrender.com/cookie'
    countries_url = 'https://country-leaders.onrender.com/countries'
    leaders_url = 'https://country-leaders.onrender.com/leaders'
    req_cookie = requests.get(cookie_url)
    cookies = cookie()
    req = requests.get(countries_url, cookies=cookies)
    countries = req.json()
    for i in countries:
        req_leaders = requests.get(leaders_url, cookies=cookies, params={'country': i})
        if req_leaders.status_code != 200:
            cookies = cookie() 
            req_leaders = requests.get(leaders_url, cookies=cookies, params={'country': i}).json()
        else:
            req_leaders = requests.get(leaders_url, cookies=cookies, params={'country': i}).json()
        leaders_dict[i] = req_leaders
    for i in countries:
        for leader in leaders_dict[i]:
            wikipedia_url = leader['wikipedia_url']
            first_paragraph = get_first_paragraph(wikipedia_url)
            leader['first_paragraph'] = first_paragraph
    return leaders_dict


Check the output of your function again.

In [ ]:
get_leaders()


Well done! It took a while however... Let's speed things up. The main *bottleneck* is the loop. We call on the Wikipedia website many times.

You will use the same *session* to call all the wikipedia pages. Check the *Advanced Usage* section of the Requests module's documentation.

Start by modifying the `get_first_paragraph()` function to accept a session parameter and adjust the `get()` method call.

In [243]:
# < 20 lines

def get_first_paragraph(wikipedia_url,session):
    print(wikipedia_url)
    first_paragraph = ''
    req_wiki = session.get(wikipedia_url).text 
    soup = bs(req_wiki, 'html.parser')
    for para in soup.find_all("p"): 
        if len(para)>8:
            first_paragraph = (para.get_text())
            #first_paragraph.append(para.get_text())
            break
    first_paragraph = re.sub(r'\[.*?\]', '', first_paragraph)
    first_paragraph = re.sub(r'\[.*?\]', '', first_paragraph)
    first_paragraph = re.sub(r'\n', ' ', first_paragraph)
    first_paragraph = re.sub(r'ⓘ', '', first_paragraph)
    first_paragraph = re.sub(r'/.*?/', '', first_paragraph)
    first_paragraph = re.sub(r'\( *;? *Écouter *\)', '', first_paragraph)
    first_paragraph = re.sub(r'Écouter', '', first_paragraph)
    first_paragraph = re.sub(r' +', ' ', first_paragraph)
    first_paragraph = re.sub(r' ,', ',', first_paragraph)
    return first_paragraph

Modify your `get_leaders()` function to make use of a single session for all the Wikipedia calls.
1. create a `Session` object outside of the loop over countries.
2. pass it to the `get_first_paragraph()` function as an argument.

In [244]:
# <25 lines
def get_leaders():
    leaders_dict = {}
    root_url = 'https://country-leaders.onrender.com/'
    status_url = 'https://country-leaders.onrender.com/status'
    cookie_url = 'https://country-leaders.onrender.com/cookie'
    countries_url = 'https://country-leaders.onrender.com/countries'
    leaders_url = 'https://country-leaders.onrender.com/leaders'
    req_cookie = requests.get(cookie_url)
    cookies = cookie()
    req = requests.get(countries_url, cookies=cookies)
    countries = req.json()
    for i in countries:
        req_leaders = requests.get(leaders_url, cookies=cookies, params={'country': i})
        if req_leaders.status_code != 200:
            cookies = cookie() 
            req_leaders = requests.get(leaders_url, cookies=cookies, params={'country': i}).json()
        else:
            req_leaders = requests.get(leaders_url, cookies=cookies, params={'country': i}).json()
        leaders_dict[i] = req_leaders
    with requests.Session() as session:
        for i in countries:
            for leader in leaders_dict[i]:
                wikipedia_url = leader['wikipedia_url']
                first_paragraph = get_first_paragraph(wikipedia_url, session)
                leader['first_paragraph'] = first_paragraph
    return leaders_dict

Test your new functions.



In [245]:
get_leaders()

https://en.wikipedia.org/wiki/George_Washington
https://en.wikipedia.org/wiki/Barack_Obama
https://en.wikipedia.org/wiki/Abraham_Lincoln
https://en.wikipedia.org/wiki/George_W._Bush
https://en.wikipedia.org/wiki/Bill_Clinton
https://en.wikipedia.org/wiki/Joe_Biden
https://en.wikipedia.org/wiki/Franklin_D._Roosevelt
https://en.wikipedia.org/wiki/Andrew_Johnson
https://en.wikipedia.org/wiki/Gerald_Ford
https://en.wikipedia.org/wiki/Richard_Nixon
https://en.wikipedia.org/wiki/Lyndon_B._Johnson
https://en.wikipedia.org/wiki/John_F._Kennedy
https://en.wikipedia.org/wiki/Dwight_D._Eisenhower
https://en.wikipedia.org/wiki/Ronald_Reagan
https://en.wikipedia.org/wiki/Harry_S._Truman
https://en.wikipedia.org/wiki/John_Adams
https://en.wikipedia.org/wiki/Thomas_Jefferson
https://en.wikipedia.org/wiki/James_Madison
https://en.wikipedia.org/wiki/James_Monroe
https://en.wikipedia.org/wiki/John_Quincy_Adams
https://en.wikipedia.org/wiki/Andrew_Jackson
https://en.wikipedia.org/wiki/Martin_Van_Buren
ht

{'us': [{'id': 'Q23',
   'first_name': 'George',
   'last_name': 'Washington',
   'birth_date': '1732-02-22',
   'death_date': '1799-12-14',
   'place_of_birth': 'Westmoreland County',
   'wikipedia_url': 'https://en.wikipedia.org/wiki/George_Washington',
   'start_mandate': '1789-04-30',
   'end_mandate': '1797-03-04',
   'first_paragraph': 'George Washington (February 22, 1732\xa0– December 14, 1799) was a Founding Father of the United States, military officer, and farmer who served as the first president of the United States from 1789 to 1797. Appointed by the Second Continental Congress as commander of the Continental Army in 1775, Washington led Patriot forces to victory in the American Revolutionary War and then served as president of the Constitutional Convention in 1787, which drafted the current Constitution of the United States. Washington has thus become commonly known as the "Father of his Country". '},
  {'id': 'Q76',
   'first_name': 'Barack',
   'last_name': 'Obama',
   

## Saving your hard work

The final step is to save the ``leaders_per_country`` dictionary in the `leaders.json` file using the [json](https://docs.python.org/3/library/json.html) module. Check out the `with` statement.

In [259]:
import json
leaders_per_country = get_leaders()
with open("leaders_per_country.json", "w") as json_file:
    json.dump(leaders_per_country, json_file)


https://en.wikipedia.org/wiki/George_Washington
https://en.wikipedia.org/wiki/Barack_Obama
https://en.wikipedia.org/wiki/Abraham_Lincoln
https://en.wikipedia.org/wiki/George_W._Bush
https://en.wikipedia.org/wiki/Bill_Clinton
https://en.wikipedia.org/wiki/Joe_Biden
https://en.wikipedia.org/wiki/Franklin_D._Roosevelt
https://en.wikipedia.org/wiki/Andrew_Johnson
https://en.wikipedia.org/wiki/Gerald_Ford
https://en.wikipedia.org/wiki/Richard_Nixon
https://en.wikipedia.org/wiki/Lyndon_B._Johnson
https://en.wikipedia.org/wiki/John_F._Kennedy
https://en.wikipedia.org/wiki/Dwight_D._Eisenhower
https://en.wikipedia.org/wiki/Ronald_Reagan
https://en.wikipedia.org/wiki/Harry_S._Truman
https://en.wikipedia.org/wiki/John_Adams
https://en.wikipedia.org/wiki/Thomas_Jefferson
https://en.wikipedia.org/wiki/James_Madison
https://en.wikipedia.org/wiki/James_Monroe
https://en.wikipedia.org/wiki/John_Quincy_Adams
https://en.wikipedia.org/wiki/Andrew_Jackson
https://en.wikipedia.org/wiki/Martin_Van_Buren
ht

Make sure the file can be read back. Write the code to read the file. And check the variables are the same.

In [260]:
with open("leaders_per_country.json", "r") as json_file:
    data = json.load(json_file)
print(data)

{'us': [{'id': 'Q23', 'first_name': 'George', 'last_name': 'Washington', 'birth_date': '1732-02-22', 'death_date': '1799-12-14', 'place_of_birth': 'Westmoreland County', 'wikipedia_url': 'https://en.wikipedia.org/wiki/George_Washington', 'start_mandate': '1789-04-30', 'end_mandate': '1797-03-04', 'first_paragraph': 'George Washington (February 22, 1732\xa0– December 14, 1799) was a Founding Father of the United States, military officer, and farmer who served as the first president of the United States from 1789 to 1797. Appointed by the Second Continental Congress as commander of the Continental Army in 1775, Washington led Patriot forces to victory in the American Revolutionary War and then served as president of the Constitutional Convention in 1787, which drafted the current Constitution of the United States. Washington has thus become commonly known as the "Father of his Country". '}, {'id': 'Q76', 'first_name': 'Barack', 'last_name': 'Obama', 'birth_date': '1961-08-04', 'death_dat

Make a function `save(leaders_per_country)` to call this code easily.

In [267]:
# 3 lines
def save(leaders_per_country):
    with open("leaders_per_country.json", "w") as json_file:
        json.dump(leaders_per_country, json_file)
    

In [269]:
leaders_per_country = get_leaders()
save(leaders_per_country)


https://en.wikipedia.org/wiki/George_Washington
https://en.wikipedia.org/wiki/Barack_Obama
https://en.wikipedia.org/wiki/Abraham_Lincoln
https://en.wikipedia.org/wiki/George_W._Bush
https://en.wikipedia.org/wiki/Bill_Clinton
https://en.wikipedia.org/wiki/Joe_Biden
https://en.wikipedia.org/wiki/Franklin_D._Roosevelt
https://en.wikipedia.org/wiki/Andrew_Johnson
https://en.wikipedia.org/wiki/Gerald_Ford
https://en.wikipedia.org/wiki/Richard_Nixon
https://en.wikipedia.org/wiki/Lyndon_B._Johnson
https://en.wikipedia.org/wiki/John_F._Kennedy
https://en.wikipedia.org/wiki/Dwight_D._Eisenhower
https://en.wikipedia.org/wiki/Ronald_Reagan
https://en.wikipedia.org/wiki/Harry_S._Truman
https://en.wikipedia.org/wiki/John_Adams
https://en.wikipedia.org/wiki/Thomas_Jefferson
https://en.wikipedia.org/wiki/James_Madison
https://en.wikipedia.org/wiki/James_Monroe
https://en.wikipedia.org/wiki/John_Quincy_Adams
https://en.wikipedia.org/wiki/Andrew_Jackson
https://en.wikipedia.org/wiki/Martin_Van_Buren
ht

## Tidy things up in a stand-alone python script

Congratulations! You now have a working scraper! However, your code is scattered throughout this notebook along side the tutorials. Hardly production ready...

Copy and paste what you need in a separate `leaders_scraper.py` file.
Make sure it works by calling `python3 leaders_scraper.py`

## To go further

If you want to practice scraping, you can read this section and tackle the exercise.

You have noticed the API returns very partial results for country leaders. Many are missing. Overwrite the `get_leaders()` function to get its list from Wikipedia and extract their *personal details* from the frame on the side.

Good luck!